# Doodle Subset Scheduler

This script takes an exported Doodle.xls file, and allows
you to filter availabiltiy for subsets of the respondents.
**Super** useful if, like me, you run a music production company
and need to schedule sectional rehearsals!


In [6]:
#Imports
import pandas as pd

In [7]:
#Read in the raw Doodle export (will implement Doodle API pulldown later.)
raw = pd.read_excel('Doodle.xls')

In [8]:
#Reformat the header so it makes sense in Pandas.
dates = raw.iloc[3]\
        .fillna(method='ffill')\
        .fillna('Names')\
        .str.split(' ')\
        .apply(lambda x: x[-1])
new_header = dates + ' | ' + raw.iloc[4].fillna('')

In [9]:
#Grab the rest of the DF and rename the columns accordingly:
df = raw.iloc[5:]
df.columns = new_header
df = df.rename(columns={'Names | ':'names'}).reset_index(drop=True)

#This bit takes care of indexing issues with NaN.
df['names'] = df['names'].fillna('')

In [12]:
#Input the names you want to filter availability for
#You can copy paste directly from Excel or Google Sheets!
names = input('Input names separated by spaces: ').split(' ')

Input names separated by spaces:  Matthew Alexander Mady Anneka NotAPerson


In [13]:
#Grab every row that matches the names you input,
#And concat them into a new DF for further processing:
res_rows = []
for name in names:
        res = df[df['names'].str.contains(name)]
        if len(res) > 0:
            res_rows.append(res)
        else:
            #If the name doesn't appear in the list, log that here.
            print("{} hasn't responded yet.".format(name))
res = pd.concat(res_rows,axis=0)

NotAPerson hasn't responded yet.


In [14]:
#Count 'OKs' in all columns, ignoring the Names column.
count_sorted = res.count().iloc[1:].sort_values(ascending=False)
#Grab the maximum shared availabilities:
max_count = max(count_sorted)

#Grab JUST the dates with the maximum number of availabilities:
best = count_sorted[count_sorted==max_count]
best

25 | 8:00 PM – 9:00 PM    4
25 | 7:00 PM – 8:00 PM    4
25 | 6:00 PM – 7:00 PM    4
24 | 7:00 PM – 8:00 PM    4
24 | 6:00 PM – 7:00 PM    4
dtype: int64

In [15]:
#Add a new column so we can sort by day number:
new_col = pd.Series(best.index.str.split(' | ')).apply(lambda x: int(x[0]))
new_col = new_col.rename(index=dict(zip(new_col.index,best.index)))
best = pd.concat([best,new_col],axis=1).rename(columns={0:'count',1:'day'})
#Sort by day number, and output to the user!
print(best.sort_values('day')[['day','count']])

                        day  count
24 | 7:00 PM – 8:00 PM   24      4
24 | 6:00 PM – 7:00 PM   24      4
25 | 8:00 PM – 9:00 PM   25      4
25 | 7:00 PM – 8:00 PM   25      4
25 | 6:00 PM – 7:00 PM   25      4
